# 尝试自定义一个线性回归模型


$y = x \cdot w +b $

假设$x \in R^{(150,4)}$, $w \in R^{(4,1)} $ , $b \in R^{(4)} $
返回 $y \in R^{(150,1)}$

In [1]:
from sklearn import datasets

iris = datasets.load_iris()

In [2]:
data = iris.data
target = iris.target

In [3]:
data.shape   # x

(150, 4)

In [4]:
target.shape #y

(150,)

# 方法1

In [8]:

import tensorflow as tf
#自定义全连接层
class Linear(tf.keras.layers.Layer):

    def __init__(self, units=1, input_dim=4):
        super(Linear, self).__init__() #
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                                  dtype='float32'), 
                             trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,),dtype='float32'),trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.constant(data) #(150,4)
linear_layer = Linear(units = 1, input_dim=4) #()
y = linear_layer(x)
print(y.shape) #(150,1)



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(150, 1)


In [ ]:
y

# 方法2

In [10]:
class Linear(tf.keras.layers.Layer):

    def __init__(self, units=1, input_dim=4):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.constant(data)
linear_layer = Linear(units = 1, input_dim=4)
y = linear_layer(x)
print(y.shape)




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(150, 1)


# 方法三

In [11]:
class Linear(tf.keras.layers.Layer):

    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape): #(150,4)
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True)
        super(Linear,self).build(input_shape)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    
    
x = tf.constant(data) #150*4
linear_layer = Linear(units = 1)
y = linear_layer(x)
print(y.shape)



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(150, 1)


# 添加不可训练的参数

In [12]:
class Linear(tf.keras.layers.Layer):

    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=False)
        super(Linear,self).build(input_shape)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    
    
x = tf.constant(data)
linear_layer = Linear(units = 1)
y = linear_layer(x)
print(y.shape)



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(150, 1)


In [11]:
print('weight:', linear_layer.weights)
print('non-trainable weight:', linear_layer.non_trainable_weights)
print('trainable weight:', linear_layer.trainable_weights)

weight: [<tf.Variable 'linear_4/Variable:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.00276536],
       [-0.07950259],
       [ 0.01646506],
       [ 0.00197834]], dtype=float32)>, <tf.Variable 'linear_4/Variable:0' shape=(1,) dtype=float32, numpy=array([0.00255805], dtype=float32)>]
non-trainable weight: [<tf.Variable 'linear_4/Variable:0' shape=(1,) dtype=float32, numpy=array([0.00255805], dtype=float32)>]
trainable weight: [<tf.Variable 'linear_4/Variable:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.00276536],
       [-0.07950259],
       [ 0.01646506],
       [ 0.00197834]], dtype=float32)>]
